### Memory

In [1]:
from langchain_core.chat_history import BaseChatMessageHistory

# 채팅 기록을 저장할 클래스 정의
# BaseChatMessageHistory를 상속 받은다.
class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []   # 메세지들을 저장할 빈 리스트 생성

    def add_ai_message(self, messages):
        # 메세지 리스트가 들어오면 기존 리스트에 추가
        self.messages.extend(messages)

    def clear(self):
        # 기록 초기화
        self.messages = []

# 세션(사용자)별로 기록을 관리할 딕셔너리 생성
# store = {'user1' : History객체, 'user2' : History객체, .....}
store = {}

# 세션 ID를 받아 해당 세션의 History 객체를 반환하는 함수
def get_by_session_id(session_id):
    if session_id not in store:
        # 새로운 InMemoryHistory 객체를 만들어서 strore에 저장
        store[session_id] = InMemoryHistory()
    # 해당하는 session_id 기록 객체를 반환
    return store[session_id]


In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 프롬프트 템플릿 생성
prompts = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill}을 잘하는 도우미야'),      # 시스템 메세지 (페르소나를 설정)
    MessagesPlaceholder(variable_name="history"),   # 대화 기록이 주입될 위치
    ('human', '{query}')                            # 사용자 질문
])

# 모델 생성
model = ChatOpenAI(model='gpt-5-nano', temperature=0.8)

# 기본 체인 생성
chain = prompts | model

# 메모리 기능 잗착한 체인 생성(RunnableWithMessageHistory)
# 기존 chain을 감싸서, 대화 기록을 자동으로 읽고 쓰는 기능 추가
chain_with_history = RunnableWithMessageHistory(
    chain, 
    get_session_history=get_by_session_id,     # 세션 ID로 기록을 찾아오는 함수 연결
    input_messages_key='query',                # 사용자 입력이 들어오는 변수명
    history_messages_key='history'             # 프롬프트에서 기록이 들어갈 변수명 (Placeholder 이름과 일치해야 함)
)

In [ ]:
response = chain_with_history.invoke(
    {'skill' : 'math', 'query' : '철수는 강아지를 세 마리 키우고 있습니다. '},
    config={'configurable' : {'session_id' : 'abc'}}     # session_id를 'abc'로 설정
)

print(response.content)

NotImplementedError in RootListenersTracer.on_chain_end callback: NotImplementedError('add_message is not implemented for this class. Please implement add_message or add_messages.')


좋아요. 3마리 강아지를 주제로 간단한 수학 문제를 몇 가지 준비해봤어요. 원하시는 난이도나 주제가 있으면 알려 주세요.

문제 1
- 세 강아지를 한 줄에 세우는 방법은 몇 가지인가?
- 정답: 6가지 (3! = 6)

문제 2
- 세 마리 중 두 마리에게 간식을 주려 한다. 서로 다른 두 마리를 고르는 방법은 몇 가지인가?
- 정답: 3가지 (C(3,2) = 3)

문제 3
- 세 마리 강아지가 서로 다른 나이를 가진다고 할 때, 무작위로 한 마리를 골르면 가장 나이가 많은 강아지가 선택될 확률은?
- 정답: 1/3

문제 4
- 세 강아지의 사료량의 비율이 1:2:3이고 총량이 600g일 때, 각 강아지가 받는 양은 얼마인가? (강아지 A=1, B=2, C=3으로 가정)
- 정답: A 100g, B 200g, C 300g

필요하시면 각 문제의 풀이 과정을 자세히 설명해 드릴게요. 다른 주제나 난이도도 말씀해 주세요.


In [ ]:
response2 = chain_with_history.invoke(
    {'skill' : 'math', 'query' : '영희가 고양이를 두 마리 키우고 있습니다.'},
    config={'configurable': {'session_id' : 'abc'}}
)

print(response2.content)

NotImplementedError in RootListenersTracer.on_chain_end callback: NotImplementedError('add_message is not implemented for this class. Please implement add_message or add_messages.')


좋아요! 두 마리 고양이를 주제로 여러 수학 문제를 함께 풀어볼 수 있어요. 어떤 종류의 문제를 원하시나요? 예를 들면 덧셈/뺄셈, 곱셈/나눗셈, 비율, 확률 등등. 간단한 예시를 바로 드릴게요.

예시 1) 몸무게 합
- 두 고양이의 몸무게를 각각 a kg, b kg이라고 두면, 총 몸무게는 a+b kg이 됩니다.
- 예: a=3 kg, b=4 kg → 총 몸무게 7 kg

예시 2) 간식 나누기 (비율)
- 두 고양이가 받는 간식의 비율이 2:3이고, 총 간식이 60 g이라고 할 때 각 고양이가 받는 양은?
- 각 비율의 합은 2+3=5이므로 한 몫은 60 g ÷ 5 = 12 g
- 고양이 A는 2몫 = 24 g, 고양이 B는 3몫 = 36 g

예시 3) 확률
- 두 마리 고양이 중 한 마리를 무작위로 고르면, 특정 고양이가 선택될 확률은 각각 1/2입니다.
- 예: 고양이 A를 선택할 확률 = 1/2, 고양이 B를 선택할 확률 = 1/2

원하시는 방향이나 숫자를 주시면, 그에 맞춰 문제를 만들어 풀어드리겠습니다. 어떤 문제를 풀어볼까요, 아니면 특정 숫자로 문제를 만들어 드릴까요?


In [18]:
# 문맥을 파악해야만 풀 수 있는 문제
response3 = chain_with_history.invoke(
    {'skill' : 'math' , 'query' :'철수랑 영희랑 합해서 몇 마리의 동물을 키우고 있나요?'},
    config={'configurable' : {'session_id' : 'abc'}}
)

print(response3.content)

NotImplementedError in RootListenersTracer.on_chain_end callback: NotImplementedError('add_message is not implemented for this class. Please implement add_message or add_messages.')


필요한 정보가 더 있습니다. 철수와 영희가 각각 몇 마리의 동물을 키우고 있는지 알려주시면 합계를 계산해 드릴게요.

계산 방법:
- 합계 = 철수의 동물 수 + 영희의 동물 수

예시:
- 철수 3마리, 영희 5마리 → 합계 3 + 5 = 8마리
- 철수 2마리, 영희 4마리, 둘 다 고양이와 강아지 포함 → 합계는 그대로 2+4 = 6마리

참고로 두 사람이 같은 동물을 공유하고 있으면 중복으로 세지지 말고 서로 다른 동물 수를 합쳐야 합니다. 숫자만 알려주시면 바로 계산해 드릴게요.


In [19]:
# 저장소 확인
print(store['abc'])